# ส่วนที่ 4.1: การวิเคราะห์และสร้างตัวอย่างการแสดงผล (Data Visualization) - ตอนที่ 1

Notebook นี้แสดงตัวอย่างการวิเคราะห์ข้อมูลและการสร้างกราฟแบบต่างๆ ที่จะนำไปสร้างใน Power BI (ส่วนที่ 1)

## 1. ติดตั้งแพ็คเกจที่จำเป็น

In [ ]:
# ติดตั้งแพ็คเกจที่จำเป็น
!pip install pandas numpy matplotlib seaborn sqlite3 sqlalchemy plotly

## 2. Import Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3
import os
from datetime import datetime
from sqlalchemy import create_engine
import plotly.express as px
import plotly.graph_objects as go
from IPython.display import display, HTML

# ตั้งค่าการแสดงผลกราฟให้สวยงาม
plt.style.use('ggplot')
sns.set(style='whitegrid')

# ตั้งค่าให้แสดงข้อมูลภาษาไทยได้
plt.rcParams['font.family'] = 'DejaVu Sans'

# แสดงเวอร์ชันของแพ็คเกจ
print(f"Pandas version: {pd.__version__}")
print(f"Matplotlib version: {plt.__version__}")
print(f"Seaborn version: {sns.__version__}")
try:
    print(f"Plotly version: {px.__version__}")
except:
    print("Plotly not installed or version info not available")

## 3. กำหนดพาธของโปรเจค

In [ ]:
# กำหนดพาธหลักสำหรับโปรเจค
project_path = './Employee_Data_Analytics_Project'  # ปรับตามโครงสร้างของคุณ

# สร้างโฟลเดอร์สำหรับเก็บรูปภาพ (ถ้ายังไม่มี)
viz_folder = f"{project_path}/visualizations"
if not os.path.exists(viz_folder):
    os.makedirs(viz_folder)
    print(f"สร้างโฟลเดอร์ {viz_folder} สำเร็จ")
else:
    print(f"โฟลเดอร์ {viz_folder} มีอยู่แล้ว")

# แสดงไฟล์ในโปรเจค
print("\nไฟล์ในโปรเจค:")
for file in os.listdir(project_path):
    if os.path.isfile(os.path.join(project_path, file)):
        print(f"- {file}")

## 4. เชื่อมต่อกับฐานข้อมูล SQLite

In [ ]:
# กำหนดพาธของฐานข้อมูล SQLite
db_path = f"{project_path}/employee_database.db"

# ตรวจสอบว่าไฟล์ฐานข้อมูลมีอยู่หรือไม่
if not os.path.exists(db_path):
    print(f"ไม่พบไฟล์ฐานข้อมูล {db_path}")
    print("กรุณารันไฟล์ 03_Data_Loading_SQLite.ipynb ก่อน")
    raise FileNotFoundError(f"ไม่พบไฟล์ฐานข้อมูล {db_path}")

# สร้างการเชื่อมต่อกับฐานข้อมูล SQLite
conn = sqlite3.connect(db_path)
print(f"เชื่อมต่อกับฐานข้อมูล {db_path} สำเร็จ")

# สร้างเอนจินสำหรับ SQLAlchemy
engine = create_engine(f'sqlite:///{db_path}')

# ตรวจสอบตารางทั้งหมดในฐานข้อมูล
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

print("\nตารางทั้งหมดในฐานข้อมูล:")
for table in tables:
    cursor.execute(f"SELECT COUNT(*) FROM {table[0]}")
    count = cursor.fetchone()[0]
    print(f"- {table[0]}: {count} แถว")

## 5. โหลดข้อมูลจากฐานข้อมูล

In [ ]:
# กำหนดชื่อตาราง
employee_table = 'employee_stats'
region_table = 'region_summary'
time_table = 'time_summary'
province_table = 'province_summary'

# โหลดข้อมูลหลัก
df_employee = pd.read_sql_query(f"SELECT * FROM {employee_table}", conn)
print(f"โหลดข้อมูลจากตาราง {employee_table} สำเร็จ: {df_employee.shape[0]} แถว, {df_employee.shape[1]} คอลัมน์")

# โหลดข้อมูลสรุปตามภูมิภาค
df_region = pd.read_sql_query(f"SELECT * FROM {region_table}", conn)
print(f"โหลดข้อมูลจากตาราง {region_table} สำเร็จ: {df_region.shape[0]} แถว, {df_region.shape[1]} คอลัมน์")

# โหลดข้อมูลสรุปตามเวลา
df_time = pd.read_sql_query(f"SELECT * FROM {time_table}", conn)
print(f"โหลดข้อมูลจากตาราง {time_table} สำเร็จ: {df_time.shape[0]} แถว, {df_time.shape[1]} คอลัมน์")

# โหลดข้อมูลสรุปตามจังหวัด
df_province = pd.read_sql_query(f"SELECT * FROM {province_table}", conn)
print(f"โหลดข้อมูลจากตาราง {province_table} สำเร็จ: {df_province.shape[0]} แถว, {df_province.shape[1]} คอลัมน์")

# แสดงข้อมูลตัวอย่าง
print("\nตัวอย่างข้อมูลหลัก:")
display(df_employee.head())

## 6. การจัดเตรียมข้อมูลเพิ่มเติมสำหรับการแสดงผล

In [ ]:
# จัดเตรียมข้อมูลเพิ่มเติมสำหรับการแสดงผล

# แปลงวันที่ให้เป็นรูปแบบที่เหมาะสม (ถ้ามี)
if 'ESTIMATED_DATE' in df_employee.columns:
    if pd.api.types.is_object_dtype(df_employee['ESTIMATED_DATE']):
        df_employee['ESTIMATED_DATE'] = pd.to_datetime(df_employee['ESTIMATED_DATE'])

# สร้างข้อมูลสรุปเพิ่มเติม
# 1. ข้อมูลสรุปตามภูมิภาคและเพศ
region_sex_summary = df_employee.groupby(['REGION', 'SEX'])['VALUE'].sum().reset_index()
region_sex_summary.rename(columns={'VALUE': 'TOTAL_EMPLOYEES'}, inplace=True)

# 2. ข้อมูลสรุปตามปีและไตรมาส
time_summary = df_employee.groupby(['YEAR', 'QUARTER', 'QUARTER_NUM'])['VALUE'].sum().reset_index()
time_summary.rename(columns={'VALUE': 'TOTAL_EMPLOYEES'}, inplace=True)
time_summary = time_summary.sort_values(['YEAR', 'QUARTER_NUM'])

# 3. ข้อมูลแนวโน้มตามเพศและเวลา
gender_time_trend = df_employee.groupby(['YEAR', 'QUARTER', 'QUARTER_NUM', 'SEX'])['VALUE'].sum().reset_index()
gender_time_trend.rename(columns={'VALUE': 'TOTAL_EMPLOYEES'}, inplace=True)
gender_time_trend = gender_time_trend.sort_values(['YEAR', 'QUARTER_NUM', 'SEX'])

# 4. ข้อมูลจังหวัดที่มีจำนวนพนักงานมากที่สุดในแต่ละภูมิภาค
top_provinces = df_employee.groupby(['REGION', 'PROVINCE'])['VALUE'].sum().reset_index()
top_provinces.rename(columns={'VALUE': 'TOTAL_EMPLOYEES'}, inplace=True)
top_provinces = top_provinces.sort_values(['REGION', 'TOTAL_EMPLOYEES'], ascending=[True, False])
top_provinces = top_provinces.groupby('REGION').head(3).reset_index(drop=True)

# แสดงข้อมูลที่จัดเตรียม
print("1. ข้อมูลสรุปตามภูมิภาคและเพศ:")
display(region_sex_summary.head())

print("\n2. ข้อมูลสรุปตามปีและไตรมาส:")
display(time_summary.head())

print("\n3. ข้อมูลแนวโน้มตามเพศและเวลา:")
display(gender_time_trend.head())

print("\n4. ข้อมูลจังหวัดที่มีจำนวนพนักงานมากที่สุดในแต่ละภูมิภาค:")
display(top_provinces.head(10))

## 7. การสร้างกราฟและการวิเคราะห์ข้อมูล

### 7.1 แผนภูมิแท่งแสดงจำนวนพนักงานตามภูมิภาค

In [ ]:
# แผนภูมิแท่งแสดงจำนวนพนักงานตามภูมิภาค
region_total = df_employee.groupby('REGION')['VALUE'].sum().reset_index()
region_total = region_total.sort_values('VALUE', ascending=False)

plt.figure(figsize=(12, 6))
ax = sns.barplot(x='REGION', y='VALUE', data=region_total, palette='viridis')
plt.title('จำนวนพนักงานรวมตามภูมิภาค', fontsize=16)
plt.xlabel('ภูมิภาค', fontsize=12)
plt.ylabel('จำนวนพนักงาน (พันคน)', fontsize=12)
plt.xticks(rotation=45)
plt.grid(axis='y', linestyle='--', alpha=0.7)

# เพิ่มค่าตัวเลขบนแผนภูมิ
for i, v in enumerate(region_total['VALUE']):
    ax.text(i, v + 5, f'{v:.1f}', ha='center', fontsize=10)

plt.tight_layout()
plt.savefig(f"{viz_folder}/region_bar_chart.png", dpi=300, bbox_inches='tight')
plt.show()

print("การวิเคราะห์: แผนภูมิแท่งเป็นเครื่องมือที่เหมาะสมสำหรับการเปรียบเทียบค่าระหว่างกลุ่มที่แตกต่างกัน (ในกรณีนี้คือภูมิภาค)")
print("- เห็นได้ชัดว่า กรุงเทพมหานคร และภาคกลาง มีจำนวนพนักงานมากที่สุด")
print("- การเรียงลำดับจากมากไปน้อยช่วยให้เห็นความแตกต่างได้ชัดเจน")
print("- แนะนำให้ใช้แผนภูมิแท่งแนวนอนใน Power BI เมื่อมีจำนวนภูมิภาคหรือหมวดหมู่มาก เพื่อให้อ่านชื่อภูมิภาคได้สะดวกขึ้น")

### 7.2 แผนภูมิวงกลมแสดงสัดส่วนพนักงานตามเพศ

In [ ]:
# แผนภูมิวงกลมแสดงสัดส่วนพนักงานตามเพศ
gender_total = df_employee.groupby('SEX')['VALUE'].sum().reset_index()
total = gender_total['VALUE'].sum()
gender_total['PERCENTAGE'] = (gender_total['VALUE'] / total * 100).round(1)

colors = ['#3498db', '#e74c3c']
plt.figure(figsize=(8, 8))
plt.pie(gender_total['VALUE'], labels=gender_total['SEX'], autopct='%1.1f%%', startangle=90, colors=colors,
        wedgeprops={'edgecolor': 'white', 'linewidth': 1.5}, textprops={'fontsize': 12})
plt.title('สัดส่วนพนักงานตามเพศ', fontsize=16)
plt.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle

# เพิ่มคำอธิบายเพิ่มเติม
legend_labels = [f"{sex}: {value:,.1f} พันคน ({pct}%)" for sex, value, pct in 
                 zip(gender_total['SEX'], gender_total['VALUE'], gender_total['PERCENTAGE'])]
plt.legend(legend_labels, loc='best', fontsize=10)

plt.savefig(f"{viz_folder}/gender_pie_chart.png", dpi=300, bbox_inches='tight')
plt.show()

print("การวิเคราะห์: แผนภูมิวงกลมเป็นเครื่องมือที่เหมาะสมสำหรับการแสดงสัดส่วนของข้อมูลเมื่อมีจำนวนหมวดหมู่น้อย (ไม่เกิน 5-7 หมวดหมู่)")
print(f"- สัดส่วนพนักงานชาย {gender_total['PERCENTAGE'].iloc[0]}% และพนักงานหญิง {gender_total['PERCENTAGE'].iloc[1]}%")
print("- แนะนำให้ใช้แผนภูมิวงกลมหรือแผนภูมิโดนัทใน Power BI สำหรับแสดงสัดส่วนที่มีจำนวนหมวดหมู่น้อย")
print("- ควรใช้สีที่แตกต่างกันอย่างชัดเจนเพื่อให้แยกแยะได้ง่าย และควรแสดงเปอร์เซ็นต์บนกราฟเพื่อให้อ่านค่าได้ทันที")

### 7.3 แผนภูมิเส้นแสดงแนวโน้มการจ้างงานตามเวลา

In [ ]:
# แผนภูมิเส้นแสดงแนวโน้มการจ้างงานตามเวลา
# สร้างข้อมูลแนวโน้มตามเวลา
time_trend = df_employee.groupby(['YEAR', 'QUARTER', 'QUARTER_NUM'])['VALUE'].sum().reset_index()
time_trend = time_trend.sort_values(['YEAR', 'QUARTER_NUM'])
time_trend['YEAR_QUARTER'] = time_trend['YEAR'].astype(str) + '-Q' + time_trend['QUARTER_NUM'].astype(str)

plt.figure(figsize=(14, 6))
plt.plot(time_trend['YEAR_QUARTER'], time_trend['VALUE'], marker='o', linewidth=2, color='#3498db')
plt.title('แนวโน้มการจ้างงานตามไตรมาส', fontsize=16)
plt.xlabel('ปี-ไตรมาส', fontsize=12)
plt.ylabel('จำนวนพนักงาน (พันคน)', fontsize=12)
plt.xticks(rotation=45)
plt.grid(True, linestyle='--', alpha=0.7)

# เพิ่มเส้นแนวโน้ม (Trend Line)
z = np.polyfit(range(len(time_trend)), time_trend['VALUE'], 1)
p = np.poly1d(z)
plt.plot(time_trend['YEAR_QUARTER'], p(range(len(time_trend))), "r--", linewidth=1)

plt.tight_layout()
plt.savefig(f"{viz_folder}/time_trend_line_chart.png", dpi=300, bbox_inches='tight')
plt.show()

print("การวิเคราะห์: แผนภูมิเส้นเป็นเครื่องมือที่เหมาะสมสำหรับการแสดงแนวโน้มตามช่วงเวลา")
print("- เห็นได้ชัดว่าจำนวนพนักงานมีการเปลี่ยนแปลงขึ้นลงตามไตรมาส")
print("- เส้นแนวโน้ม (เส้นประสีแดง) แสดงให้เห็นแนวโน้มโดยรวมของการจ้างงาน")
print("- แนะนำให้ใช้แผนภูมิเส้นใน Power BI สำหรับการแสดงข้อมูลแนวโน้มตามช่วงเวลา")

### 7.4 แผนภูมิเส้นแสดงแนวโน้มการจ้างงานแยกตามเพศ

In [ ]:
# แผนภูมิเส้นแสดงแนวโน้มการจ้างงานแยกตามเพศ
gender_time = gender_time_trend.copy()
gender_time['YEAR_QUARTER'] = gender_time['YEAR'].astype(str) + '-Q' + gender_time['QUARTER_NUM'].astype(str)

plt.figure(figsize=(14, 6))
for sex, color in zip(['ชาย', 'หญิง'], ['#3498db', '#e74c3c']):
    data = gender_time[gender_time['SEX'] == sex]
    plt.plot(data['YEAR_QUARTER'], data['TOTAL_EMPLOYEES'], marker='o', linewidth=2, label=sex, color=color)

plt.title('แนวโน้มการจ้างงานตามเพศและไตรมาส', fontsize=16)
plt.xlabel('ปี-ไตรมาส', fontsize=12)
plt.ylabel('จำนวนพนักงาน (พันคน)', fontsize=12)
plt.xticks(rotation=45)
plt.grid(True, linestyle='--', alpha=0.7)
plt.legend(fontsize=12)

plt.tight_layout()
plt.savefig(f"{viz_folder}/gender_time_trend_line_chart.png", dpi=300, bbox_inches='tight')
plt.show()

print("การวิเคราะห์: แผนภูมิเส้นหลายเส้นช่วยให้เห็นการเปรียบเทียบแนวโน้มระหว่างกลุ่มต่างๆ (ในที่นี้คือเพศ)")
print("- สามารถเปรียบเทียบแนวโน้มการจ้างงานระหว่างเพศชายและหญิงได้อย่างชัดเจน")
print("- เห็นได้ว่าแนวโน้มการจ้างงานของทั้งสองเพศมีรูปแบบคล้ายกัน แต่มีจำนวนที่แตกต่างกัน")
print("- แนะนำให้ใช้แผนภูมิเส้นหลายเส้นใน Power BI สำหรับการเปรียบเทียบแนวโน้มระหว่างกลุ่มต่างๆ")

### 7.5 แผนภูมิแท่งแบบจัดกลุ่มแสดงการเปรียบเทียบจำนวนพนักงานตามเพศและภูมิภาค

In [ ]:
# แผนภูมิแท่งแบบจัดกลุ่มแสดงการเปรียบเทียบจำนวนพนักงานตามเพศและภูมิภาค
region_sex = df_employee.groupby(['REGION', 'SEX'])['VALUE'].sum().reset_index()
region_sex = region_sex.pivot(index='REGION', columns='SEX', values='VALUE').reset_index()
region_sex = region_sex.sort_values('ชาย', ascending=False)

plt.figure(figsize=(14, 8))
x = np.arange(len(region_sex['REGION']))
width = 0.35

fig, ax = plt.subplots(figsize=(14, 8))
rects1 = ax.bar(x - width/2, region_sex['ชาย'], width, label='ชาย', color='#3498db')
rects2 = ax.bar(x + width/2, region_sex['หญิง'], width, label='หญิง', color='#e74c3c')

ax.set_title('จำนวนพนักงานตามเพศและภูมิภาค', fontsize=16)
ax.set_xlabel('ภูมิภาค', fontsize=12)
ax.set_ylabel('จำนวนพนักงาน (พันคน)', fontsize=12)
ax.set_xticks(x)
ax.set_xticklabels(region_sex['REGION'], rotation=45)
ax.legend(fontsize=12)
ax.grid(axis='y', linestyle='--', alpha=0.7)

# เพิ่มค่าตัวเลขบนแผนภูมิ
def autolabel(rects):
    for rect in rects:
        height = rect.get_height()
        ax.annotate(f'{height:.1f}',
                    xy=(rect.get_x() + rect.get_width() / 2, height),
                    xytext=(0, 3),  # 3 points vertical offset
                    textcoords="offset points",
                    ha='center', va='bottom', fontsize=9)

autolabel(rects1)
autolabel(rects2)

fig.tight_layout()
plt.savefig(f"{viz_folder}/region_sex_bar_chart.png", dpi=300, bbox_inches='tight')
plt.show()

print("การวิเคราะห์: แผนภูมิแท่งแบบจัดกลุ่มช่วยให้เห็นการเปรียบเทียบระหว่างกลุ่มย่อยในแต่ละหมวดหมู่หลัก")
print("- สามารถเปรียบเทียบจำนวนพนักงานชายและหญิงในแต่ละภูมิภาคได้อย่างชัดเจน")
print("- เห็นได้ว่าในทุกภูมิภาคมีจำนวนพนักงานชายมากกว่าพนักงานหญิง")
print("- แนะนำให้ใช้แผนภูมิแท่งแบบจัดกลุ่มใน Power BI สำหรับการเปรียบเทียบระหว่างกลุ่มย่อยในแต่ละหมวดหมู่หลัก")